### Expression Quantification Pipeline: TPM Calculation, SRR-Merging, and Cell-Line-Level Aggregation

This code performs the full post-alignment quantification workflow in three stages:

---

## **1. Per-SRR calculation of RPK, TPM, and log2(TPM+1)**

For each SRR experiment listed in `../srr_ids.txt`, the script:

1. Loads gene-level count tables generated by 6_get_genes_counts.sh: `../counts_<TYPE>/<SRR>gene_counts<TYPE>.tsv`



where `<TYPE>` is either `"unique"` or `"fraction"`.

2. Renames the final column to `counts`.

3. Keeps the essential columns:
- `Geneid`
- `gene_name`
- `Length`
- `counts`

4. Computes: <br>
**RPK (Reads Per Kilobase):**

$$
\text{RPK} = \frac{\text{counts}}{\text{Length} / 1000}
$$

**TPM (Transcripts Per Million):**

$$
\text{TPM} = \frac{\text{RPK}}{\sum \text{RPK}} \times 10^6
$$

**log₂(TPM+1)**

5. Writes output to:

`../expression_metrics_<TYPE>/<SRR>_expression_metrics.tsv`


### **Input for Stage 1**
- `../srr_ids.txt` — list of SRR IDs  
- Count tables from featureCounts:  
`../counts_unique/<SRR>_gene_counts_unique.tsv`  
`../counts_fraction/<SRR>_gene_counts_fraction.tsv`  
Each must contain:
- `Geneid`
- `gene_name`
- `Length`
- raw counts (last column)

### **Output for Stage 1**
Directory:  
../expression_metrics_unique/

../expression_metrics_fraction/

Files:  
`<SRR>_expression_metrics.tsv` containing columns:  
`Geneid, gene_name, Length, counts, RPK, TPM, log2(TPM+1)`

---

## **2. Merge per-SRR expression into gene × SRR matrices**

For both `unique` and `fraction` modes, and for each metric:




The script:

1. Reads each SRR expression file:  
   `../expression_metrics_<TYPE>/<SRR>_expression_metrics.tsv`

2. Extracts:  
   `Geneid`, `gene_name`, `<metric>`  
   and renames `<metric>` → `<SRR>`.

3. Performs an **inner merge** across all SRRs to ensure gene consistency.

4. Saves matrices into: `expression_tables_<TYPE>/<metric>_exps.tsv`



### **Output for Stage 2**
Expression matrices of shape **genes × SRRs**, e.g.:<br>
expression_tables_unique_MANE_mRNA/TPM_exps.tsv <br>
expression_tables_fraction_MANE_mRNA/counts_exps.tsv



---

## **3. Merge expression values by cell line (replicate averaging)**

Using metadata (`GSE240542_SraRunTable.csv`), the code groups SRRs by cell line.

For each metric in:

counts, RPK, TPM


and for each TYPE in `unique`, `fraction`:

1. Loads SRR-level expression matrix.
2. Computes the mean across replicates belonging to the same cell line.
3. Saves: `expression_tables_<TYPE>/<metric>_mean_cell_lines.tsv`



4. For TPM, also computes log2(mean(TPM)+1)

and stores: `expression_tables_<TYPE>/log2(TPM+1)_mean_cell_lines.tsv`


### **Output for Stage 3**
Cell-line-level matrices:  
- `counts_mean_cell_lines.tsv`
- `RPK_mean_cell_lines.tsv`
- `TPM_mean_cell_lines.tsv`
- `log2(TPM+1)_mean_cell_lines.tsv` (TPM only)

---

# **Pipeline Summary**

### **Inputs**
- Per-gene count files from featureCounts  
- SRR list (`srr_ids.txt`)  
- MANE-filtered expression metric files  
- Metadata linking SRR → cell line  

### **Outputs**
- Per-SRR TPM tables  
- Gene × SRR expression matrices  
- Gene × cell line matrices  

The code below performs the entire computation workflow.


In [ ]:
import os 
import pandas as pd
from pathlib import Path
import numpy as np

# --- 1. Per-SRR calculation of expression metrics (RPK, TPM, log2(TPM+1)) ---

# fraction / unique
TYPE = 'unique'
output = f'../expression_metrics_{TYPE}'

# path to directory with per-SRR gene count tables
base_dir = f"../counts_{TYPE}"

# read list of SRR IDs
srr_ids = []
with open('../srr_ids.txt', 'r') as file:
    srr_ids = [i.strip() for i in file.readlines()]

for idd in srr_ids:
    # load featureCounts-derived table: one file per SRR
    counts_table = pd.read_csv(f'{base_dir}/{idd}_gene_counts_{TYPE}.tsv', sep='\t')

    # the last column is raw read counts
    counts_table.rename(columns={counts_table.columns[-1]: 'counts'}, inplace=True)

    # keep only required columns
    counts_table = counts_table[['Geneid', 'gene_name', 'Length', 'counts']].copy()

    # RPK: Reads Per Kilobase
    counts_table['RPK'] = counts_table['counts'] / (counts_table['Length'] / 1000.0)

    # total RPK across all genes (normalization factor for TPM)
    sum_RPK = counts_table['RPK'].sum()

    # TPM: Transcripts Per Million
    counts_table['TPM'] = (counts_table['RPK'] / sum_RPK) * 1e6

    # log-transformed TPM
    counts_table['log2(TPM+1)'] = np.log2(counts_table['TPM'] + 1)

    # save per-SRR expression metrics
    counts_table.to_csv(f'{output}/{idd}_expression_metrics.tsv', sep='\t', index=False)


# --- 2. Merge by experiments (build gene × SRR matrices) ---

# fraction / unique
TYPES = ['unique', 'fraction']
# TYPE = 'unique'

# metrics to aggregate
# counts / RPK / TPM / log2(TPM+1)
all_metrics = ['counts', 'RPK', 'TPM', 'log2(TPM+1)']
# metric_of_interest = 'counts'

# build expression matrices for each TYPE and each metric
for TYPE in TYPES:
    output_dir = f'expression_tables_{TYPE}_MANE_mRNA'
    # input directory with per-SRR expression metrics (MANE protein-coding subset)
    expr_dir = f'../expression_metrics_{TYPE}_for_MANE_protein_coding_only'
    
    for metric_of_interest in all_metrics:
    
        merged_df = pd.DataFrame()
        
        for srr in srr_ids:
            # per-SRR expression metrics file
            tsv_path = os.path.join(expr_dir, f'{srr}_expression_metrics.tsv')
            df = pd.read_csv(tsv_path, sep='\t')

            # keep only gene ID, gene name and the metric of interest
            df = df[['Geneid', 'gene_name', metric_of_interest]].copy()

            # rename metric column to SRR ID (will become sample column)
            df = df.rename(columns={metric_of_interest: srr})
        
            if merged_df.empty:
                # first SRR: initialize merged_df
                merged_df = df
            else:
                # subsequent SRRs: inner join on (Geneid, gene_name)
                merged_df = pd.merge(merged_df, df, on=['Geneid', 'gene_name'], how='inner')
                
        # save gene × SRR matrix for this metric and TYPE
        merged_df.to_csv(f'{output_dir}/{metric_of_interest}_exps.tsv', sep='\t', index=False)


# --- 3. Merge by cell line (average replicates per cell line) ---

# load metadata: must contain at least 'cell_line' and 'Run' (SRR IDs)
metadata = pd.read_csv('GSE240542_SraRunTable.csv')

# list of unique cell lines
cell_lines = metadata['cell_line'].unique()

# map: cell_line -> list of SRR IDs (replicates)
lines_to_srr = {}
for line in cell_lines:
    exps = list(metadata[metadata['cell_line'] == line]['Run'])
    lines_to_srr[line] = exps

# log2(TPM+1) should not be averaged directly across SRRs
# use metrics: counts / RPK / TPM
all_metrics = ['counts', 'RPK', 'TPM']
# metric_of_interest = 'counts'

# fraction / unique
TYPES = ['unique', 'fraction']
# TYPE = 'unique'

for TYPE in TYPES:
    output_dir = f'expression_tables_{TYPE}_MANE_mRNA'

    for metric_of_interest in all_metrics:
        # load gene × SRR expression matrix
        merged_df = pd.read_csv(f'{output_dir}/{metric_of_interest}_exps.tsv', sep='\t')
        
        # new DataFrame for gene × cell_line
        replics_mean = pd.DataFrame()
        replics_mean['Geneid'] = merged_df['Geneid']
        replics_mean['gene_name'] = merged_df['gene_name']
        
        # compute mean across SRRs belonging to the same cell line
        for line, srr_list in lines_to_srr.items():
            replics_mean[line] = merged_df[srr_list].mean(axis=1)
        
        # save mean expression per cell line (for this metric and TYPE)
        replics_mean.to_csv(f'{output_dir}/{metric_of_interest}_mean_cell_lines.tsv', sep='\t', index=False)

        # For TPM: compute log2(mean(TPM)+1) at the cell-line level
        if metric_of_interest == 'TPM':
            replics_mean_log = replics_mean.copy()
            numeric_cols = replics_mean_log.select_dtypes(include=[np.number]).columns
            replics_mean_log[numeric_cols] = np.log2(replics_mean_log[numeric_cols] + 1)
            replics_mean_log.to_csv(f'{output_dir}/log2(TPM+1)_mean_cell_lines.tsv', sep='\t', index=False)
